In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    os.chdir('/content/drive/Othercomputers/내 노트북/gdrive/dppos/shfe')
    !pip uninstall -y gym
    !pip install gymnasium stable-baselines3
except ModuleNotFoundError:
    print('코랩아님')

텐서보드 log 확인

```
tensorboard --logdir ./logs/
```

[http://localhost:6006](http://localhost:6006)

In [ ]:
import time
from drivingppo.common import set_seed
from drivingppo.environment import WorldEnv
from drivingppo.model import train_start, train_resume, linear_schedule, evaluate
from drivingppo.model import (
    NoFeaturesExtractor,
    SharedWpFeaturesExtractor,
    Shwp216FeaturesExtractor,
)
from world_samples import gen_1, gen_2t as gen_2, gen_3t as gen_3, gen_3l, generate_random_world_plain

import numpy as np
from IPython.display import clear_output
pi = np.pi

prefix = 'shfe'
t0 = time.strftime('%y%m%d-%H%M')

policy_kwargs=dict(
    features_extractor_class=Shwp216FeaturesExtractor,
    features_extractor_kwargs=dict(),
    net_arch=dict(
        pi=[512, 512],
        vf=[512, 512, 256]
    )
)

def a(seed):
    tb_log = True
    run_name = f'{prefix}-{t0}-{seed}'

    model = train_start(  # 모델 생성만
        gen_env=lambda: WorldEnv(),
        policy_kwargs=policy_kwargs,
        steps=0,
        seed=seed
    )

    passed = False
    count = 0
    while not passed:
        count += 1
        print(f'=== 1단계 {count} ===')
        set_seed(seed)
        model = train_resume(
            model=model,
            save_path=f'{prefix}-{seed}--1',
            gen_env=lambda: WorldEnv(world_generator=gen_1, time_gain_limit=9_000, collision_ending=False),
            tb_log=tb_log,
            run_name=run_name,
            steps=1_0000,
            lr=9e-4,
            gamma=0.9,
            seed=seed
        )
        clear_output(wait=True)
        metrics = evaluate(model, gen_1, csv_path='', episode_num=100)
        endings = metrics['ending/type']
        success_rate = (endings.count('arrived') + endings.count('timeout')) / len(endings)
        if success_rate >= 0.5 - 0.0001:
            passed = True
            print(f'=== 1단계 통과({count}). (성공률: {success_rate*100:.0f}%)')
        elif count > 10:
            passed = True
            print(f'=== 1단계 통과 못함. (성공률: {success_rate*100:.0f}%) 그만')
        else:
            print(f'=== 1단계 통과 못함. (성공률: {success_rate*100:.0f}%) 다시')
    if count > 10:
        return

    passed = False
    count = 0
    while not passed:
        count += 1
        print(f'=== 2단계 {count} ===')
        set_seed(seed)
        model = train_resume(
            model=model,
            save_path=f'{prefix}-{seed}--2',
            gen_env=lambda: WorldEnv(world_generator=gen_2, collision_ending=False),
            tb_log=tb_log,
            run_name=run_name,
            steps=1_0000,
            lr=5e-4,
            gamma=0.99,
            seed=seed
        )
        clear_output(wait=True)
        metrics = evaluate(model, gen_2, csv_path='', episode_num=100)
        endings = metrics['ending/type']
        success_rate = (endings.count('arrived') + endings.count('timeout')) / len(endings)
        if success_rate >= 0.5 - 0.0001:
            passed = True
            print(f'=== 2단계 통과({count}). (성공률: {success_rate*100:.0f}%)')
        elif count > 10:
            passed = True
            print(f'=== 2단계 통과 못함. (성공률: {success_rate*100:.0f}%) 그만')
        else:
            print(f'=== 2단계 통과 못함. (성공률: {success_rate*100:.0f}%) 다시')
    if count > 10:
        return

    print('=== 3단계 ===')
    set_seed(seed)
    model = train_resume(
        model=model,
        gen_env=lambda: WorldEnv(world_generator=gen_3),
        tb_log=tb_log,
        run_name=run_name,
        save_freq=10_0000,
        steps=200_0000,
        lr=linear_schedule(3e-4),
        gamma=1,
        seed=seed
    )
    model = train_resume(  # 학습률 스케줄러 안 쓰도록 모델 저장
        model=model,
        save_path=f'{prefix}-{seed}--3',
        gen_env=lambda: WorldEnv(world_generator=gen_3),
        steps=0,
        lr=0.0,
    )
    clear_output(wait=True)
    evaluate(f'{prefix}-{seed}--3', gen_3l, episode_num=100, csv_path='', verbose=True)

a(100)
a(200)
a(300)
a(400)
a(500)

In [ ]:
# 텐서보드로그 재깍재깍 안 나옴.
try:
    from google.colab import drive
    drive.flush_and_unmount()
except ModuleNotFoundError:
    print('코랩아님')

In [ ]:
# 학습률 스케줄 쓰면 코랩에서 돌린 게 내 컴에서 실행이 안 됨. 그거만 덮어쓰고 다시 저장.
from stable_baselines3 import PPO

path = 'checks/wwwwwwwwwwwwwww'

model = PPO.load(
    path=path,
    learning_rate=0.0,
)
model.save(path)

| `log_std` | 실제 표준편차 σ | 탐색 수준 | 비고 |
| --- | --- | --- | --- |
| 0 | 1.0 | 매우 높음 | 행동 공간 전체를 휘젓고 다니는 수준. 초기 학습 시 강한 탐색이 필요할 때 사용. |
| -0.5 | ≈ 0.6 | 높음 | 행동의 변동폭이 큼. 기존 정책이 굳었을 때 강제 리셋용으로 권장되는 값. |
| -1 | ≈ 0.37 | 보통 | 안정적인 주행과 탐색이 공존하는 수준. |
| -2 | ≈ 0.13 | 낮음 | 어느 정도 최적화가 진행된 상태. 정밀한 제어를 시작함. |
| \~-3.0 | \< 0.05 | 거의 없음 | 수렴 완료 단계. 거의 결정론적(Deterministic)인 행동을 보임. |